In [1]:
import pandas as pd
from pathlib import Path

## 1. Cargar los datos 

In [2]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
# Carpeta padre (un nivel arriba)
PARENT_DIR = BASE_DIR.parent

# Directorios de datos
directorio_13 = PARENT_DIR / "13. Mallado Potencia Instalada"
directorio_3 = PARENT_DIR / "03. Datos_Climaticos"

# Carga de datos mallado
df_mall_pot = pd.read_csv(f"{directorio_13}/MALLADO_Potencia.csv", sep=',', encoding='latin-1')
df_mall_pob = pd.read_csv(f"{directorio_13}/MALLADO_Población.csv", sep=',', encoding='latin-1')
df_mall_clima = pd.read_csv(f"{directorio_3}/pvgis_diario_2019_2023_europe_madrid_20251102_1034.csv", sep=',', encoding='latin-1')


## 2. Introducir la key en clima

In [3]:
# Cambiamos nombre columnas
df_mall_clima = df_mall_clima.rename(columns={'lat': 'LAT_CENTRO', 'lon': 'LON_CENTRO'})


# Obtenemos las tres columnas de tiempo: año, mes, día
df_mall_clima['date_local'] = pd.to_datetime(df_mall_clima['date_local'], errors='coerce')

# Extrae año, mes (1–12) y día (1–31)
df_mall_clima['Year'] = df_mall_clima['date_local'].dt.year
df_mall_clima['Month'] = df_mall_clima['date_local'].dt.month
df_mall_clima['Day'] =df_mall_clima['date_local'].dt.day

# Añadimos la columna cell_id
df_mall_clima = df_mall_clima.merge(
    df_mall_pob[['cell_id', 'LAT_CENTRO', 'LON_CENTRO']],
    on=['LAT_CENTRO', 'LON_CENTRO'],
    how='left'
)

# Hacemos merge de potencia instalada
df_mall_clima = df_mall_clima.merge(
    df_mall_pot[['cell_id', 'Year', 'Pot_Solar_kW', 'Pot_Eolico_kW']],
    on=['cell_id', 'Year'],
    how='left'
)

# Hacemos merge de población
df_mall_clima = df_mall_clima.merge(
    df_mall_pob[['cell_id', 'POBLACION_TOTAL']],
    on=['cell_id'],
    how='left'
)

#Elegimos columnas finales
columnas_finales = ['cell_id', 'Year', 'Month', 'Day', 'Temp_min_C', 'Temp_max_C', 'Temp_media_C', 'RadiacionGlobal_media_Wm2', 'Viento_media_ms', 'Pot_Solar_kW', 'Pot_Eolico_kW', 'POBLACION_TOTAL', 'LAT_CENTRO', 'LON_CENTRO']
df_mall_total=[]
df_mall_total = df_mall_clima[columnas_finales]
#

In [4]:
# Guardar resultados
df_mall_total.to_csv(f"{BASE_DIR}/MALLADO_TOTAL.csv", index=False)